In [ ]:
import numpy as np
import cv2
import sqlite3
#tkinter is used for the GUI. 
from tkinter import *
import tkinter 
from PIL import Image, ImageTk
import time
from datetime import date
#Produce sound
import winsound
import random
#Multithreading
import threading
from playsound import playsound
import matplotlib.pyplot as plt
import pandas as pd



#engine = pyttsx.init()
#engine.say('Good morning.')
#engine.runAndWait()

#Parameters for alert sound
frequency=2500
duration=1000
#winsound.Beep(frequency, duration)
db = sqlite3.connect("my_database.sql") #you can put whatever ... created if not exist
conn = db.cursor()

switch=True
#Number of times the driver has slept
noOfSleeps=0

#Load dataset
eyescascade = cv2.CascadeClassifier('haarcascade_eye.xml')

#Flag used to kill the opencv thread
flag=False

#Configure GUI
window = Tk()
window.configure(bg="black")

l = Label (window, text="Drowsiness Alert System", bg="black", fg="white")
l.grid(row=0,column=0)
cap=None
e = None
wordcol=None
t2=None
lab=None
lab1 = None
lab2 = None
reclabel=None
stopbut=None
b=Button(window, text="Submit", width=15, height=2)
#Number or threads running
threadCount=0
def check():
    global e
    global switch
    global lab
    global b
    global lab1
    global lab2
    choice=e.get().lower()
    print(choice)
    if choice == wordcol:
        print('correct')
        switch=False
        lab.destroy()
        e.destroy()
        b.destroy()
        lab1.destroy()
        lab2.destroy()
    else:
        print('wrong')
        
        
        
        

def minigame():
    global switch
    global e
    global wordcol
    global t2
    global lab
    global lab1
    switch=True
    global window
    global lab2
    global b
    t2 = threading.Thread(target = AlertSound)
    t2.start()
    color=["white","red","green","orange"]
    random.shuffle(color)
    word=color[0]
    wordcol=color[1]
    lab2=Label (window, text="You are probably drowsy!", bg="black", fg="white", font=('Helvetica',20))
    lab2.grid(row=8,column=0)
    time.sleep(0.5)
    lab1=Label (window, text="Enter the colour of the word to turn the alarmn off", bg="black", fg="white")
    lab1.grid(row=9,column=0)
    lab = Label (window, text=word, bg="black", fg=wordcol,font = ('Helvetica', 30))
    lab.grid(row=10,column=0)
    e = Entry(window, width=50)
    e.grid(row=11,column=0)
    b = Button(window, text="Submit", width=15, height=2, command=check)
    b.grid(row=12,column=0)
    window.update()
    
    
    
def AlertSound():
    print("alert")
    global frequency
    global duration
    global switch
    while switch:
        #winsound.Beep(frequency, duration)
        playsound('alarm.mp3')
        print('ha')
        #time.sleep(5)
        
        
    
    



#Method to start journey
def start():
    global cap
    global flag
    flag=False
    global threadCount
    global reclabel
    
    if threadCount==0:
        threadCount=1
        #Get video footage for opencv from webcam
        cap = cv2.VideoCapture(0)
        #Create a thread for the opencv process        
        t1 = threading.Thread(target = show_frame)
        #Begin thread execution
        t1.start()
        reclabel = Label(window,text='Journey Started',bg='black',fg='red')
        reclabel.grid(row=0,column=10)
        window.update()
        
     
        

#Method to stop journey
def stop():
    
    global flag
    global noOfSleeps
    global threadCount
    global reclabel
    #Alter flag value so that we can terminate the thread
    flag=True
    print("stopped")
    if threadCount>0:
        reclabel.destroy()
    #Deinitialize the thread
    t1=None
    threadCount=0
    #Release webcam from process
    cap.release()
    Label (window, text="You slept "+str(noOfSleeps)+" time(s)", bg="black", fg="white").grid(row=5,column=0)
    window.update()


#Method to detect drowsiness
def show_frame():
    global t2
    #Number of times eyes are closed
    closedcnt=0
    #Get and process webcam footage
    while cap.isOpened():
        _, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        eyes = eyescascade.detectMultiScale(gray,1.1,4)
        numOfeyes=0
        for (x,y,w,h) in eyes:
            cv2.rectangle(img, (x,y), (x+w,y+h),(255,0,0),1)
            numOfeyes+=1
        #cv2.imshow('your face',img)
        font = cv2.FONT_HERSHEY_SIMPLEX
        global noOfSleeps
        if numOfeyes == 0:
            if closedcnt > 1:
                cv2.putText(img,'WAKE UP!',(100,50), font, 4, (255, 255, 255), 2, cv2.LINE_AA)
                print('WAKE UP!')
                
                noOfSleeps+=1
                minigame()
                t2.join()
                cnlosedcnt=0
            else:
                closedcnt+=1
        else:
            closedcnt=0
        print(numOfeyes)
        
        global flag
        
        #If flag(kill process) = True, come out of the function
        if flag:
            flag=False  
            return
        
        time.sleep(1)
def record():
    

    conn.execute("CREATE TABLE IF NOT EXISTS Activities (timestamp int, sleeps int);")
    r = conn.execute("SELECT * from Activities where timestamp=? ",(date.today(),))
    rec = conn.fetchall()
    print (str(rec)+'  '+str(len(rec)))
    if len(rec)==0:
        conn.execute("INSERT INTO Activities (timestamp,sleeps) VALUES (?,?)",(date.today(),noOfSleeps))
        db.commit()
    else:
        
        old = rec[0]
        new = int(old[1]) + noOfSleeps
        print(old)
        conn.execute("UPDATE Activities SET sleeps=? where timestamp=?",(new,date.today()))
        db.commit()
def showrecord(): 
    conn.execute("SELECT * FROM Activities")
    r = conn.fetchall()
    print(r)
    labelcounter=7
    for rec in r:
          
        dt=str(rec[0])
        ns=str(rec[1])
        Label (window, text=dt+"->"+ns, bg="black", fg="white").grid(row=labelcounter,column=0)
        window.update()
        labelcounter+=1
          
def emptyrecord():
    conn.execute("DROP TABLE Activities;")
    db.commit()
    
def viz():
    conn.execute("SELECT timestamp,sleeps FROM Activities group by timestamp having sleeps=max(sleeps)")
    r = conn.fetchall()
    dates=[]
    values=[]
    n=len(r)-1
    for rec in r:

        dates.append(str(rec[0]))
        values.append(str(rec[1]))
    false_x = [x for x in range(len(dates))]
    plt.xlabel('Dates')
    plt.ylabel('No. of times slept')

    plt.plot(false_x,values, 'o-')
    
    plt.xticks(range(len(dates)), (dates), rotation=20)
    #plt.axis([xmin, xmax, 0, 100]) #- sets axes limits on graph
    plt.axis([-1, 8, 0, 11])
    #plt.show()
    plt.savefig('drowsiness_chart.png')
    load = Image.open("drowsiness_chart.png")
    render = ImageTk.PhotoImage(load)
    
    



#Create buttons  
Button(window, text="Start Journey", width=15, height=2, command=start).grid(row=2,column=0)
Button(window, text="Stop Journey", width=15, height=2, command=stop).grid(row=2,column=2)
Button(window, text="End Day", width=15, height=2, command=record).grid(row=2,column=3)
Button(window, text="Show Records", width=15, height=2, command=showrecord).grid(row=2,column=4)
Button(window,text="Drowsiness graph",width=15, height=2, command=viz).grid(row=2,column=5)
#Only for debugging
Button(window, text="Delete All Data", width=15, height=2, command=emptyrecord).grid(row=2,column=6)
  
    
#Display GUI  
window.mainloop()

0
0
2
0
2
2
2
2
2
1
1
0
1
1
1
2
1
1
1
1
0
2
2
2
2
0
1
1
0
1
1
1
1
0
1
1
1
0
0
WAKE UP!
alert
ha
red
wrong
ha
green
correct
ha
0
2
2
stopped
[]  0
[('2020-01-31', 2), ('2020-03-15', 1)]
